In [1]:
import os
import openpyxl
import pandas as pd
from PyPDF2 import PdfReader
import ipywidgets as widgets
import numpy as np
from IPython.display import display
from datetime import datetime
from ipywidgets import interact
from openpyxl import Workbook

In [2]:
path_excel_mov = r"C:\Users\OneEngInst\Projetos_python\dashboard\content\mov_ac.xls"
path_excel_comp =r"C:\Users\OneEngInst\Projetos_python\notebook\OEM\content\lista_componente.xlsx"

In [3]:
# Diretório onde estão os PDFs
directory = r"C:\Users\OneEngInst\OneDrive - ONE ENGENHARIA INDUSTRIA E COMERCIO LTDA\001_INTELBRAS\AC\1_0-CORRETIVA\1ªTRANCHE\2024\01T_AC_COR_REPOSITORIO"
bdsip = r"C:\Users\OneEngInst\Projetos_python\dashboard\content\base_sip_Concluido.xlsx"
# Lista para armazenar os dados
data = []

# Função para extrair o valor após um termo específico
def extract_value(text, term):
    if term in text:
        start_index = text.find(term) + len(term)
        # Aqui consideramos que o valor pode estar logo abaixo do termo, em uma nova linha
        value = text[start_index:].splitlines()[1].strip()
        return value
    return None

# Percorrer todos os arquivos no diretório
for filename in os.listdir(directory):
    if filename.endswith(".pdf"):
        filepath = os.path.join(directory, filename)
        reader = PdfReader(filepath)
        print(f"Lendo arquivo: {filename}")
        
        # Dicionário para armazenar os valores extraídos de cada PDF
        pdf_data = {"Arquivo": filename}
        
        # Percorrer todas as páginas do PDF
        for page in reader.pages:
            text = page.extract_text()
            
            # Extrair os valores desejados
            sigfi = extract_value(text, "SIGFI:")
            num_os = extract_value(text, "NUM OS:")
            
            # Armazenar os valores no dicionário
            if sigfi:
                pdf_data["SIGFI"] = sigfi
            if num_os:
                pdf_data["NUM OS"] = num_os

        # Adicionar o dicionário ao conjunto de dados, se tiver valores relevantes
        if pdf_data.get("SIGFI") or pdf_data.get("NUM OS"):
            data.append(pdf_data)

Lendo arquivo: 641703_100633_CARLOS_COSTA_DOS_SANTOS_14604.pdf
Lendo arquivo: 642332_115323_CLAUDECY_DOS_SANTOS_DE_MENEZES_13529.pdf
Lendo arquivo: 642484_101410_JOSE_MARIA_NUNES_DA_COSTA_13536.pdf
Lendo arquivo: 642558_103708_MARIA_JARLENE_TAVARES_DA_COSTA_13746.pdf
Lendo arquivo: 642558_116531_MARIA_JARLENE_TAVARES_DA_COSTA_13746.pdf
Lendo arquivo: 642567_103693_MARIANA_GOMES_DO_NASCIMENTO_13404.pdf
Lendo arquivo: 642567_115750_MARIANA_GOMES_DO_NASCIMENTO_13404.pdf
Lendo arquivo: 642753_104106_MARIA_DAISIA_LIMA_DE_SOUZA_14606.pdf
Lendo arquivo: 642764_105481_VANUZIA_SILVA_DE_CARVALHO_13730.pdf
Lendo arquivo: 642764_116297_VANUZIA_SILVA_DE_CARVALHO_13730.pdf
Lendo arquivo: 642769_109323_FRANCISCO_COSTA_TAVARES_13750.pdf
Lendo arquivo: 642864_99855_MARIA_LUZIA_DUTRA_DA_SILVA_13752.pdf
Lendo arquivo: 642939_102510_FRANCISCO_GELIDO_FIRMINO_MIRANDA_13498.pdf
Lendo arquivo: 642941_101482_JOSE_GOMES_FIRMINO_13483.pdf
Lendo arquivo: 642946_103703_JOSE_FRANCISCO_DA_COSTA_13524.pdf
Lendo arqui

In [6]:
df_filtered_global_oem = None

def ler_planilha():
    """Lê uma planilha do Excel."""
    excluir_status = ['TREINAMENTO','DUPLICADO','CANCELADO ']
    df_bd = pd.read_excel(r"C:\Users\OneEngInst\Projetos_python\dashboard\content\base_sip_Concluido.xlsx")
    df = df_bd[~df_bd['STATUS'].isin(excluir_status)]
    return df

def aplicar_filtros(rota_value, status_value, tipo_value, executor_value, origem_value, tipocausa_value, data_inicial, data_final):
    """Aplica filtros a uma planilha."""
    global df_filtered_global_oem
    df_filtered_global_oem = df[
        df["ROTA"].isin(rota_value) &
        df["STATUS"].isin(status_value) &
        df["TIPO"].isin(tipo_value) &
        df["EXECUTOR"].isin(executor_value) &
        df["ORIGEM"].isin(origem_value) &
        df["TIPOCAUSA"].isin(tipocausa_value) &
        (df["DATACONCLUSAO"] >= data_inicial) &
        (df["DATACONCLUSAO"] <= data_final)
    ]
    return df_filtered_global_oem

if __name__ == "__main__":
    df = ler_planilha()

    df["DATACONCLUSAO"] = pd.to_datetime(df["DATACONCLUSAO"]).dt.date

    rota_options = sorted(list(df['ROTA'].unique()))
    rota_filter = widgets.SelectMultiple(options=rota_options, description="Rotas:")

    status_options = sorted(list(df['STATUS'].unique()))
    status_filter = widgets.SelectMultiple(options=status_options, description="Status:")

    tipo_options = sorted(list(df['TIPO'].unique()))
    tipo_filter = widgets.SelectMultiple(options=tipo_options, description="Tipo:")

    executor_options = sorted(list(df['EXECUTOR'].unique()))
    executor_filter = widgets.SelectMultiple(options=executor_options, description="Usuário:")

    origem_options = sorted(list(df['ORIGEM'].unique()))
    origem_filter = widgets.SelectMultiple(options=origem_options, description="Origem:")

    tipocausa_options = sorted(list(df['TIPOCAUSA'].unique()))
    tipocausa_filter = widgets.SelectMultiple(options=tipocausa_options,descrition="Tipo Causa")

    data_inicial_filter = widgets.DatePicker(value=df["DATACONCLUSAO"].min())
    data_final_filter = widgets.DatePicker(value=df["DATACONCLUSAO"].max())

    aplicar_filter_button = widgets.Button(description="Aplicar filtros")

    def on_button_click(event):
        global df_filtered_global_oem
        df_filtrado = aplicar_filtros(
            rota_filter.value, status_filter.value, tipo_filter.value,
            executor_filter.value, origem_filter.value,
            tipocausa_filter.value,
            data_inicial_filter.value, data_final_filter.value
        )
        df_filtered_global_oem = df_filtrado

    aplicar_filter_button.on_click(on_button_click)

    display(data_inicial_filter)
    display(data_final_filter)
    df_filtered_global_oem
    display(
        widgets.HBox([origem_filter, tipo_filter, rota_filter, executor_filter, status_filter],
                     layout=widgets.Layout(align_items='baseline'))
    )
    display(
        widgets.HBox([tipocausa_filter, aplicar_filter_button],
                     layout=widgets.Layout(align_items='center'))
    )

DatePicker(value=datetime.date(1900, 1, 1), step=1)

DatePicker(value=datetime.date(2024, 9, 18), step=1)

In [7]:
df_filtered_global_oem.shape

(862, 55)

In [8]:
df_table =df_filtered_global_oem.rename(columns={'IDSIGFI':'CÓDIGO ONE','UC': 'NÚM.DA UC','OBRA':'NÚM. DA OBRA','NOMECLIENTE':'NOME DO CLIENTE','CPF':'CPF','ENDERECO':'ENDEREÇO',
                                                   'MUNICIPIO':'MUNICÍPIO','TIPOCAUSA':'CORRETIVA','EXECUTOR':'EQUIPE EXECUTORA','OBRA':'NÚM.DA OBRA','ROTA':'ROTA','TIPODESIGFI': 'TIPO/SIGFI',
                                                    'DTCONCLUSAO':'DATA ATENDIMENTO'})
                                                     

replacements1 = {'ONEOM001':'EQ.1','ONEOM002':'EQ.2','ONEOM003':'EQ.3','ONEOM004':'EQ.4','ONEOM005':'EQ.5','ONEOM006':'EQ.6','ONEOM007':'EQ.7',
                 'ONEOM008':'EQ.8','ONEOM009':'EQ.9','ONEOM010':'EQ.10','ONEOM011':'EQ.11','ONEOM012':'EQ.12','ONEOM013':'EQ.13','ONEOM014':'EQ.14',
                 'ONEOM015':'EQ.15'}
df_table['EQUIPE EXECUTORA'] = df_table['EQUIPE EXECUTORA'].replace(replacements1)

replacements2 = {'INTERMITENICA':'CORRETIVA'}
df_table['CORRETIVA'] = df_table['CORRETIVA'].replace(replacements2)

replacements3 = {1:'CRUZEIRO DO SUL', 2:'CRUZEIRO DO SUL', 3:'CRUZEIRO DO SUL',
                 4:'CRUZEIRO DO SUL', 5:'CRUZEIRO DO SUL', 6:'CRUZEIRO DO SUL',
                 7:'CRUZEIRO DO SUL', 8:'CRUZEIRO DO SUL', 9:'CRUZEIRO DO SUL',
                 10:'CRUZEIRO DO SUL',13:'CRUZEIRO DO SUL',12:'BRASILÉIA',
                 11:'RIO BRANCO', 14:'CRUZEIRO DO SUL', 15:'CRUZEIRO DO SUL', 
                 16:'CRUZEIRO DO SUL', 17:'CRUZEIRO DO SUL',21:'TARAUACÁ',
                 22:'CRUZEIRO DO SUL',23:'TARAUACÁ', 24:'CRUZEIRO DO SUL'}

df_table['REGIONAL'] = df_table['ROTA'].replace(replacements3)

df_table = df_table[['CÓDIGO ONE', 'NÚM.DA UC', 'NÚM.DA OBRA', 'NOME DO CLIENTE', 'CPF',
                       'ENDEREÇO', 'MUNICÍPIO','REGIONAL', 'DATA ATENDIMENTO', 'CORRETIVA', 'ROTA',
                       'EQUIPE EXECUTORA', 'TIPO/SIGFI', 'NUMOS']]

In [9]:
df_movement = pd.read_excel(path_excel_mov)
df_componentlist = pd.read_excel(path_excel_comp)
df_date = df_filtered_global_oem[['NUMOS','NUMOCORRENCIA','EXECUTOR','STATUS', 'LATLONCON','DTAHORARECLAMACAO','DTCONCLUSAO','CONCLUSAO','DESCADICIONALPROG']]
table_dt = df_date.copy()
table_dt['DTAHORARECLAMACAO'] = pd.to_datetime(table_dt['DTAHORARECLAMACAO'],errors='coerce')
table_dt['DTCONCLUSAO'] = pd.to_datetime(table_dt['DTCONCLUSAO'])
table_dt['MES ABERTURA'] = table_dt['DTAHORARECLAMACAO'].dt.month
table_dt['ANO ABERTURA'] = table_dt['DTAHORARECLAMACAO'].dt.year
table_dt['MES CONCLUSAO'] = table_dt['DTCONCLUSAO'].dt.month
table_dt['ANO CONCLUSAO'] = table_dt['DTCONCLUSAO'].dt.year
table_dt['HORA CONCLUSAO'] = table_dt['DTCONCLUSAO'].dt.strftime('%H:%M')
#dataset[['LATITUDE', 'LONGITUDE','ERRO_1','ERRO_2']] = dataset['LATLONCON'].apply(lambda x: pd.Series(x.split(',')))#,'ERRO_1','ERRO_2'
def split_latlon(x):
    if isinstance(x, str):
        return pd.Series(x.split(','))
    else:
        #Se não for uma string, retorna NaN ou valores padrão, dependendo do que você deseja
        return pd.Series([pd.NaT, pd.NaT])

# Aplica a função split_latlon à coluna 'LATLONCON'
table_dt[['LATITUDE', 'LONGITUDE']] = table_dt['LATLONCON'].apply(split_latlon)#,'erro_1','erro_2'

replacements_1 = {'ONEOM001':'EQ.1','ONEOM002':'EQ.2','ONEOM003':'EQ.3','ONEOM004':'EQ.4','ONEOM005':'EQ.5','ONEOM006':'EQ.6','ONEOM007':'EQ.7',
                 'ONEOM008':'EQ.8','ONEOM009':'EQ.9','ONEOM010':'EQ.10','ONEOM011':'EQ.11','ONEOM012':'EQ.12','ONEOM013':'EQ.13','ONEOM014':'EQ.14',
                 'ONEOM015':'EQ.15'}
table_dt['EQUIPE'] = table_dt['EXECUTOR'].replace(replacements_1)

# Calculando as colunas data de fechamento e abertura
today = datetime.now()
#Aplica a formula para função do excell =SE(AD921="";HOJE()-AA921;AD921-AD921)
def days_of_late(row):
    if pd.isnull(row['DTCONCLUSAO']):
        return today - row['DTCONCLUSAO']
    else:
        return row['DTCONCLUSAO'] - row['DTAHORARECLAMACAO']
        days = diff.days
        hours = diff.seconds // 3600
        return f'{days} dias e {hours} horas'
#Aplicando a função
table_dt['CONCLUSAO - ABERTURA'] = table_dt.apply(days_of_late, axis=1)

In [10]:
table_dt

,NUMOS,NUMOCORRENCIA,EXECUTOR,STATUS,LATLONCON,DTAHORARECLAMACAO,DTCONCLUSAO,CONCLUSAO,DESCADICIONALPROG,MES ABERTURA,ANO ABERTURA,MES CONCLUSAO,ANO CONCLUSAO,HORA CONCLUSAO,LATITUDE,LONGITUDE,EQUIPE,CONCLUSAO - ABERTURA
68,202400069706,117551,ONEOM024,VALIDADO,"-8.04547166,-72.759408",2024-09-06 10:39:00,2024-09-07 14:15:00,NORMALIZADO SEM SUBST DE EQUIPAMENTO,"EM INSPEÇÃO REALIZADA NESTA DATA, FORAM IDENTI...",9,2024,9,2024,14:15,-8.04547166,-72.759408,ONEOM024,1 days 03:36:00
76,202400069615,116590,ONEOM024,VALIDADO,"-7.873686666666667,-72.72774666666666",2024-09-05 00:00:00,2024-09-08 14:20:00,NORMALIZADO SEM SUBST DE EQUIPAMENTO,NaN,9,2024,9,2024,14:20,-7.873686666666667,-72.72774666666666,ONEOM024,3 days 14:20:00
77,202400069607,116546,ONEOM024,VALIDADO,"-8.040463333333333,-72.77069333333334",2024-09-05 00:00:00,2024-09-07 12:49:00,NORMALIZADO COM SUBST DE EQUIPAMENTO,NaN,9,2024,9,2024,12:49,-8.040463333333333,-72.77069333333334,ONEOM024,2 days 12:49:00
78,202400069599,116531,ONEOM024,VALIDADO,"-8.088445,-72.77528666666667",2024-09-05 10:01:00,2024-09-06 11:53:00,NORMALIZADO SEM SUBST DE EQUIPAMENTO,NaN,9,2024,9,2024,11:53,-8.088445,-72.77528666666667,ONEOM024,1 days 01:52:00
82,202400069540,116535,ONEOM024,VALIDADO,"-8.038278333333333,-72.74834333333334",2024-09-05 13:51:00,2024-09-06 13:16:00,NORMALIZADO SEM SUBST DE EQUIPAMENTO,NaN,9,2024,9,2024,13:16,-8.038278333333333,-72.74834333333334,ONEOM024,0 days 23:25:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9798,202200024406,74058,ONEOM004,VALIDADO,"-8.330065,-71.17998166666666",2024-05-10 14:25:00,2024-05-17 10:47:00,NORMALIZADO COM SUBST DE EQUIPAMENTO,MANUTENÇÃO CORRETIVA,5,2024,5,2024,10:47,-8.330065,-71.17998166666666,EQ.4,6 days 20:22:00
9817,202200021972,72643,ONEOM007,VALIDADO,"-7.8734011,-73.05490743",2024-05-07 16:00:00,2024-05-27 16:36:00,NORMALIZADO COM SUBST DE EQUIPAMENTO,MANUTENÇÃO CORRETIVA,5,2024,5,2024,16:36,-7.8734011,-73.05490743,EQ.7,20 days 00:36:00
9818,202200021881,69250,ONEOM007,VALIDADO,"-7.87270715,-73.07313507",2024-04-30 13:50:00,2024-05-27 15:31:00,NORMALIZADO COM SUBST DE EQUIPAMENTO,MANUTENÇÃO CORRETIVA,4,2024,5,2024,15:31,-7.87270715,-73.07313507,EQ.7,27 days 01:41:00
9819,202200021758,71562,ONEOM007,VALIDADO,"-7.86296136,-72.99371107",2024-05-05 13:00:00,2024-05-28 09:01:00,NORMALIZADO COM SUBST DE EQUIPAMENTO,MANUTENÇÃO CORRETIVA,5,2024,5,2024,09:01,-7.86296136,-72.99371107,EQ.7,22 days 20:01:00


In [11]:
component_install = df_movement.loc[(df_movement['TIPO1'] =='INSTALADO')]
# Criando uma função para realizar a contagem de componentes
def count_component(component_install):
    descrition = []

    for idservico in component_install['NUMOS'].unique():
        subset_component = component_install[component_install['NUMOS'] == idservico]        
        count_component = subset_component['DESCRICAO'].value_counts()
        format_descrition = ', '.join(f'Troca de {count} {component}' if count > 1 else f'Troca de 1 {component}' for component, count in count_component.items())
        descrition.append(format_descrition)

    df_descrition= pd.DataFrame({'NUMOS':component_install['NUMOS'].unique(), 'DESCRICAO_FINAL':descrition})
    return df_descrition
observation =  count_component(component_install)
table_dt = table_dt.merge(observation, left_on='NUMOS', right_on='NUMOS', how='left')

In [12]:
table_dt 

,NUMOS,NUMOCORRENCIA,EXECUTOR,STATUS,LATLONCON,DTAHORARECLAMACAO,DTCONCLUSAO,CONCLUSAO,DESCADICIONALPROG,MES ABERTURA,ANO ABERTURA,MES CONCLUSAO,ANO CONCLUSAO,HORA CONCLUSAO,LATITUDE,LONGITUDE,EQUIPE,CONCLUSAO - ABERTURA,DESCRICAO_FINAL
0,202400069706,117551,ONEOM024,VALIDADO,"-8.04547166,-72.759408",2024-09-06 10:39:00,2024-09-07 14:15:00,NORMALIZADO SEM SUBST DE EQUIPAMENTO,"EM INSPEÇÃO REALIZADA NESTA DATA, FORAM IDENTI...",9,2024,9,2024,14:15,-8.04547166,-72.759408,ONEOM024,1 days 03:36:00,NaN
1,202400069615,116590,ONEOM024,VALIDADO,"-7.873686666666667,-72.72774666666666",2024-09-05 00:00:00,2024-09-08 14:20:00,NORMALIZADO SEM SUBST DE EQUIPAMENTO,NaN,9,2024,9,2024,14:20,-7.873686666666667,-72.72774666666666,ONEOM024,3 days 14:20:00,NaN
2,202400069607,116546,ONEOM024,VALIDADO,"-8.040463333333333,-72.77069333333334",2024-09-05 00:00:00,2024-09-07 12:49:00,NORMALIZADO COM SUBST DE EQUIPAMENTO,NaN,9,2024,9,2024,12:49,-8.040463333333333,-72.77069333333334,ONEOM024,2 days 12:49:00,Troca de 1 Bateria MSL 100/24V
3,202400069599,116531,ONEOM024,VALIDADO,"-8.088445,-72.77528666666667",2024-09-05 10:01:00,2024-09-06 11:53:00,NORMALIZADO SEM SUBST DE EQUIPAMENTO,NaN,9,2024,9,2024,11:53,-8.088445,-72.77528666666667,ONEOM024,1 days 01:52:00,NaN
4,202400069540,116535,ONEOM024,VALIDADO,"-8.038278333333333,-72.74834333333334",2024-09-05 13:51:00,2024-09-06 13:16:00,NORMALIZADO SEM SUBST DE EQUIPAMENTO,NaN,9,2024,9,2024,13:16,-8.038278333333333,-72.74834333333334,ONEOM024,0 days 23:25:00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
857,202200024406,74058,ONEOM004,VALIDADO,"-8.330065,-71.17998166666666",2024-05-10 14:25:00,2024-05-17 10:47:00,NORMALIZADO COM SUBST DE EQUIPAMENTO,MANUTENÇÃO CORRETIVA,5,2024,5,2024,10:47,-8.330065,-71.17998166666666,EQ.4,6 days 20:22:00,Troca de 2 Bateria MSL 100/24V
858,202200021972,72643,ONEOM007,VALIDADO,"-7.8734011,-73.05490743",2024-05-07 16:00:00,2024-05-27 16:36:00,NORMALIZADO COM SUBST DE EQUIPAMENTO,MANUTENÇÃO CORRETIVA,5,2024,5,2024,16:36,-7.8734011,-73.05490743,EQ.7,20 days 00:36:00,Troca de 1 Bateria MSL 100/24V
859,202200021881,69250,ONEOM007,VALIDADO,"-7.87270715,-73.07313507",2024-04-30 13:50:00,2024-05-27 15:31:00,NORMALIZADO COM SUBST DE EQUIPAMENTO,MANUTENÇÃO CORRETIVA,4,2024,5,2024,15:31,-7.87270715,-73.07313507,EQ.7,27 days 01:41:00,Troca de 1 Bateria MSL 100/24V
860,202200021758,71562,ONEOM007,VALIDADO,"-7.86296136,-72.99371107",2024-05-05 13:00:00,2024-05-28 09:01:00,NORMALIZADO COM SUBST DE EQUIPAMENTO,MANUTENÇÃO CORRETIVA,5,2024,5,2024,09:01,-7.86296136,-72.99371107,EQ.7,22 days 20:01:00,Troca de 2 Bateria MSL 100/24V


In [13]:
datareader = pd.DataFrame(data)
datareader["NUM OS"] = datareader["NUM OS"].astype('int64')
numocorrencia = datareader.rename(columns={"NUM OS":"NUMOS"})

In [14]:
table_two = numocorrencia.merge(table_dt, on='NUMOS', how='left')
table_one = table_two.merge(df_table, on='NUMOS', how='left')

In [15]:
plan_one = table_one[['CÓDIGO ONE', 'NÚM.DA UC', 'NÚM.DA OBRA', 'NOME DO CLIENTE',
                       'CPF', 'ENDEREÇO', 'MUNICÍPIO', 'REGIONAL', 'DATA ATENDIMENTO',
                        'CORRETIVA', 'ROTA', 'EQUIPE EXECUTORA', 'TIPO/SIGFI', 'LATITUDE', 
                        'LONGITUDE','NUMOS']]
plan_two = table_two[['NUMOCORRENCIA', 'DTAHORARECLAMACAO','MES ABERTURA','ANO ABERTURA','DTCONCLUSAO','CONCLUSAO - ABERTURA', 
                      'MES CONCLUSAO','ANO CONCLUSAO', 'HORA CONCLUSAO','EQUIPE','STATUS', 'LATITUDE', 
                      'LONGITUDE', 'CONCLUSAO','DESCRICAO_FINAL','DESCADICIONALPROG']]
plan_two['DESCRICAO_FINAL'].replace('NORMALIZADO SEM SUBST DE EQUIPAMENTO', 'NORMALIZADO COM SUBST DE EQUIPAMENTO')
plan_two.loc[(plan_two['CONCLUSAO'] == 'NORMALIZADO SEM SUBST DE EQUIPAMENTO') | 
             (plan_two['CONCLUSAO'] == 'REINSTALACAO EXECUTADA'), 'DESCRICAO_FINAL'] = 'Não houve troca de componentes'

In [16]:
plan_two

,NUMOCORRENCIA,DTAHORARECLAMACAO,MES ABERTURA,ANO ABERTURA,DTCONCLUSAO,CONCLUSAO - ABERTURA,MES CONCLUSAO,ANO CONCLUSAO,HORA CONCLUSAO,EQUIPE,STATUS,LATITUDE,LONGITUDE,CONCLUSAO,DESCRICAO_FINAL,DESCADICIONALPROG
0,100633,2024-07-25 16:30:00,7,2024,2024-08-02 10:41:00,7 days 18:11:00,8,2024,10:41,EQ.3,VALIDADO,-8.087651666666666,-72.777395,NORMALIZADO COM SUBST DE EQUIPAMENTO,Troca de 1 Bateria MSL 100/24V,NaN
1,115323,2024-09-01 00:00:00,9,2024,2024-09-05 11:35:00,4 days 11:35:00,9,2024,11:35,EQ.3,VALIDADO,-7.70756951,-72.00697219,NORMALIZADO COM SUBST DE EQUIPAMENTO,Troca de 2 Bateria MSL 100/24V,NaN
2,101410,2024-07-28 17:20:00,7,2024,2024-08-01 12:12:00,3 days 18:52:00,8,2024,12:12,ONEOM024,VALIDADO,-7.720478333333333,-71.99774666666667,NORMALIZADO COM SUBST DE EQUIPAMENTO,Troca de 2 Bateria MSL 100/24V,NaN
3,103708,2024-08-05 08:27:00,8,2024,2024-08-07 14:18:00,2 days 05:51:00,8,2024,14:18,ONEOM024,VALIDADO,-8.088461666666666,-72.77519666666667,NORMALIZADO COM SUBST DE EQUIPAMENTO,Troca de 1 Bateria MSL 100/24V,NaN
4,116531,2024-09-05 10:01:00,9,2024,2024-09-06 11:53:00,1 days 01:52:00,9,2024,11:53,ONEOM024,VALIDADO,-8.088445,-72.77528666666667,NORMALIZADO SEM SUBST DE EQUIPAMENTO,Não houve troca de componentes,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,101073,2024-07-30 19:28:00,7,2024,2024-08-04 11:10:00,4 days 15:42:00,8,2024,11:10,EQ.15,VALIDADO,-8.254745000000002,-70.96741999999999,NORMALIZADO COM SUBST DE EQUIPAMENTO,"Troca de 1 Bateria MSL 100/24V, Troca de 1 Inv...",O CLIENTE SOLICITOU O ATENDIMENTO NO DIA 27/07...
140,105403,2024-08-08 11:12:00,8,2024,2024-08-22 16:22:00,14 days 05:10:00,8,2024,16:22,ONEOM024,VALIDADO,-7.925528,-72.767641,NORMALIZADO COM SUBST DE EQUIPAMENTO,"Troca de 2 Bateria MSL 100/24V, Troca de 1 Con...",MANUTENÇÃO CORRETIVA
141,102037,2024-07-30 14:31:00,7,2024,2024-08-02 13:09:00,2 days 22:38:00,8,2024,13:09,EQ.15,VALIDADO,-8.266003333333334,-71.056715,NORMALIZADO SEM SUBST DE EQUIPAMENTO,Não houve troca de componentes,NaN
142,113450,2024-08-26 00:00:00,8,2024,2024-08-27 17:37:00,1 days 17:37:00,8,2024,17:37,EQ.7,VALIDADO,-9.19176699,-72.71240443,NORMALIZADO COM SUBST DE EQUIPAMENTO,Troca de 1 Bateria MSL 100/24V,NaN


In [17]:
# Criar uma planilha base
wb = Workbook()
ws = wb.create_sheet('producao_corretiva')

# Escrever os dados do plan_one
row_offset = 1
for col_idx, col in enumerate(plan_one.columns, start=1):
    ws.cell(row=row_offset, column=col_idx, value=col)

for r_idx, row in plan_one.iterrows():
    for col_idx, value in enumerate(row, start=1):
        ws.cell(row=row_offset + r_idx + 1, column=col_idx, value=value)

# Atualizar o deslocamento para a próxima coluna
column_offset = plan_one.shape[1] + 2

# Escrever os dados do plan_two ao lado do plan_one
for col_idx, col in enumerate(plan_two.columns, start=column_offset):
    ws.cell(row=1, column=col_idx, value=col)

for r_idx, row in plan_two.iterrows():
    for col_idx, value in enumerate(row, start=column_offset):
        ws.cell(row=r_idx + 2, column=col_idx, value=value)

# Salvar a planilha
wb.save(r"C:\Users\OneEngInst\Projetos_python\notebook\OEM\output\1ºtrache_corretiva_lot_21ªremessa_intelbras_ac.xlsx")